In [ ]:
import re
import json
from openai import OpenAI
from dotenv import load_dotenv
from langchain import PromptTemplate
load_dotenv()
client = OpenAI()

In [ ]:
test_src_path = "data/raw/recipes/dev.src"
def load_from_lines(path):
    with open(path, 'r') as f:
        return [line.strip() for line in f]
test_src = load_from_lines(test_src_path)

In [ ]:
target_path = "my_output/recipes/gpt-3.5-turbo/gen/dev"

In [ ]:
prompt_template = PromptTemplate.from_template("You have a recipe with a masked section. Please fill in the missing part of the recipe covered by [MASK]. Note that [MASK] can be several sentences split by \n. Output only the text that you generate for the [MASK], not the whole recipe. Do not include any [MASK] token in your answer. Recipe: {recipe}. \n [MASK]:")

In [ ]:
completions = []
for i in range(1000):
  prompt = prompt_template.format(recipe=test_src[i])
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You generate sentences masked by the [MASK] token."},
      {"role": "user", "content": prompt}
    ],
    n=10,
    temperature=1,
  )
  print([c.message.content for c in completion.choices])
  completions.append([c.message.content for c in completion.choices])

In [ ]:
# create target path if doesnt exist
import os
if not os.path.exists(target_path):
    os.makedirs(target_path)
for i in range(10):
    split = [c[i] for c in completions]
    with open(f"{target_path}/{i}.gen", 'w') as f:
        f.write("\n".join(split))